In [1]:
# clique = (0, 1, 2)

# [(a, b) for a in clique for b in clique if a != b]

In [2]:
# from ortools.linear_solver import pywraplp as mip

# # unaries
# A1 = 0.9
# A2 = 0.9
# A3 = 0.9

# B1 = 0.9
# B2 = 0.9
# B3 = 0.9

# solver = mip.Solver('t', mip.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
# Sum = []
# Rho = {}
# V = [A1, A2, A3, B1, B2, B3]
# n = len(V)
# for idx in range(n):
#     Rho[idx] = solver.BoolVar('rho[%i]' % idx)
    

# s = solver.Sum(
#     Rho[idx] * V[idx] for idx in range(n))
# Sum.append(s)

# E = [(0, 1, 2), (3, 4)]

# for clique in E:
#     #s = solver.Sum(Rho[a] for a in clique for b in clique if a != b)
#     s = solver.Sum(Rho[a] * Rho[b] for a in clique for b in clique if a != b)
    
# solver.Maximize(solver.Sum(Sum))
# RESULT = solver.Solve()
# print("Time = ", solver.WallTime(), " ms")
# print("result:", RESULT)
# print('\nTotal cost:', solver.Objective().Value())

# for idx in range(n):
#     print('node ' + str(idx) + ' -> ', Rho[idx].solution_value())

In [12]:
from ortools.linear_solver import pywraplp as mip
import numpy.linalg as la
import numpy as np
import networkx as nx

# unaries
A1 = 1
A2 = 1
A3 = 1

B1 = 1
B2 = 1
B3 = 1

C1 = 1
C2 = 1
C3 = 1

V = [A1, A2, A3, B1, B2, B3, C1, C2, C3]

E_c = [(0, 3, 6), (4, 7)]
E_v = [(0, 1, 2), (3, 4, 5), (6, 7, 8)]

# --------------------------------
solver = mip.Solver('t', mip.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Sum = []
Rho = {}
Chi = {}
n = len(V)
for idx in range(n):
    Rho[idx] = solver.BoolVar('rho[%i]' % idx)

s = solver.Sum(
    Rho[idx] * V[idx] for idx in range(n))
Sum.append(s)

for clique in E_c:
    solver.Add(solver.Sum(Rho[node] for node in clique) <= 1)

for clique in E_v:
    for a in clique:
        for b in clique:
            if a >= b:
                continue
            Chi[a, b] = solver.BoolVar('chi[%i,%i]' % (a, b))
    
    s = solver.Sum(Chi[a, b] * 1 for a in clique for b in clique if b > a)
    Sum.append(s)
    
    for a in clique:
        for b in clique:
            if a >= b:
                continue
            solver.Add(2 * Chi[a, b] <= Rho[a] + Rho[b])
    
solver.Maximize(solver.Sum(Sum))
RESULT = solver.Solve()
print("Time = ", solver.WallTime(), " ms")
print("result:", RESULT)
print('\nTotal cost:', solver.Objective().Value())

for idx in range(n):
    print('node ' + str(idx) + ' -> ', Rho[idx].solution_value())

print('\n===========================')
for (a,b), v in Chi.items():
    print(str((a, b)) + ' -> ', v.solution_value())

Time =  7  ms
result: 0

Total cost: 10.0
node 0 ->  1.0
node 1 ->  1.0
node 2 ->  1.0
node 3 ->  0.0
node 4 ->  0.0
node 5 ->  1.0
node 6 ->  0.0
node 7 ->  1.0
node 8 ->  1.0

(0, 1) ->  1.0
(0, 2) ->  1.0
(1, 2) ->  1.0
(3, 4) ->  0.0
(3, 5) ->  0.0
(4, 5) ->  0.0
(6, 7) ->  0.0
(6, 8) ->  0.0
(7, 8) ->  1.0
